# Automotive Data analyser

- Phase 1: CSV files with the format `ABSOLUTE_TIME, ID, BINARY`
- Phase 2: CSV files with the format `ABSOLUTE_TIME, can_id_var, can_id_var, ...` where `can = {0,1}`, `id` is a unique identifier for the source and `var` is the identifier of the decoded field.

## Contents

- [Initial setup](#Initial-setup)
- [Preprocessing](#Preprocessing)
  - [Load binary data frames](#Load-binary-data-frames)
  - [Calculate interarrival packet times for each ID and CANline](#Calculate-interarrival-packet-times-for-each-ID-and-CANline)
  - [Calculate bitflips and bitflips magnitude](#Calculate-bitflips-and-bitflips-magnitude)
- [Data field decode](#Data-field-decode)
  - [Data statistics](#Data-statistics)
  - [Datablock Analysis](#Datablocks-analysis)
- [Export figures](#Export-figures)
  - [Packet count per ID and CANline](#Packet-count-per-ID-and-CANline)
  - [Interrarrival packet time distributions and boxplots](#Interrarrival-packet-time-distributions-and-boxplots)
  - [Bitflips heatmaps](#Bitflips-heatmaps)
  
Once the preprocess is complete, you may skip directly to the decode section to speed-up the computation

## Initial setup


In [ ]:
from IPython.display import display
from fastprogress import master_bar, progress_bar
import seaborn as sns
import matplotlib.pyplot as plt
import os
from datetime import datetime
import numpy as np
import statistics 
import math
import pickle
import scipy.stats as scstat            
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import matplotlib.ticker as ticker
import matplotlib.gridspec as gridspec

# If true exports vectorial PDFs instead of JPG.
VECTORIAL_FIGURES = False
FIG_EXTENSION = "pdf" if VECTORIAL_FIGURES else "jpg"

ROOT_DIR = "absolute-path-to-project-root-folder"
VEHICLE = "vechile/experiment"
DATA_DIR = ROOT_DIR + "Data/" + VEHICLE + "/"
GRAPHICS_DIR = ROOT_DIR + "Graphics/" + VEHICLE + "/" + FIG_EXTENSION + "/"

try:
    os.makedirs(GRAPHICS_DIR)
except FileExistsError:
    # directory already exists
    pass

# Either use pandas or modin, not both
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_rows = None

# Configure RAY and MODIN
#import modin.pandas as mpd

## Preprocessing

### Load binary data frames
Import the raw file, rename the columns accordingly and parse the timestamp as index.
This step can be skipped if the dataframe has been already processed and stored as indicated below.

In [ ]:
%%time
# Load the CSV
df = pd.read_csv(DATA_DIR + "raw.csv", 
                 #sep= r' +|,|\t', # Files may have one or more spaces as separator (also include commas and tabs).
                 sep=",",
                 dtype={0:object, 1:object, 2:object, 3:object, 4:object}, 
                 #nrows=1000,
                 header=None
                )

df.rename(columns={0:'time', 1:'CAN', 2:'id', 3:'bytes', 4:'binary'}, inplace=True)
df.columns = map(str.lower, df.columns)


# Convert timestamp column to datetime index
timestamps = np.around(df['time'].astype(float), 6).apply(datetime.fromtimestamp)
datetimes = pd.to_datetime(timestamps)
idx = pd.DatetimeIndex(datetimes, freq='infer').copy(deep=True)
#df.set_index(idx, inplace=True)
#df.drop('time', axis=1, inplace=True)
df['time'] = idx

# Remove support variables
del(timestamps)
del(datetimes)
del(idx)

# Convert CAN line and ID columns to categorical data
df['can'] = pd.Categorical(df['can'])
df['id'] = pd.Categorical(df['id'])

In [ ]:
%%time
# Save the intermediate result as Pickle file to speedup future analysis
#df.to_pickle(DATA_DIR + "raw.pkl")

In [ ]:
display(df.info())
display(df.can.unique())
display(df.id.unique())
display(df.head())

### Calculate interarrival packet times for each ID and CANline

For each uid in the RAW data, extract the interarrival packet time

In [ ]:
%%time

df.loc[:,'timedelta'] = np.nan
df.loc[:,'timedelta_ms'] = np.nan

grp = df.groupby(by=['id','can'])
for (uid, can), group in progress_bar(grp):     
    try:
        diffs = group.time.diff().dropna()
        diffs_ms = np.multiply(diffs.dt.total_seconds(),1000)

        df.loc[diffs.index, 'timedelta'] = diffs
        df.loc[diffs_ms.index, 'timedelta_ms'] = diffs_ms

    except Exception as ex:
        print("EXCEPTION!!", uid, can, ex)
        display(group)
        break


# Drop first element of each (CAN,ID) pair which will be NaT anyway
#data = data[np.isfinite(data['timedelta_ms'])]
df.dropna(subset=['timedelta_ms'], inplace=True)

# Add a log transform to the ms field
df.loc[:,'timedelta_ms_log'] = np.log10(df['timedelta_ms'])

In [ ]:
display(df.info())
display(df.can.unique())
display(df.id.unique())
display(df.head())

In [ ]:
%%time
# Save the intermediate result as Pickle file to speedup future analysis
#df.to_pickle(DATA_DIR + "raw.interrarrivaltimes.pkl")

### Calculate bitflips and bitflips magnitude

Calculate the bitflips for each bit in the frame, grouped by ID and CANline.

In [ ]:
%%time

def create_bitflips(data):    
    import math
    
    bitflips = pd.DataFrame()
    bitflips['id'] = ""
    bitflips['can'] = ""
    
    magnitudes = pd.DataFrame()
    magnitudes['id'] = ""
    magnitudes['can'] = ""

    gprs = data.groupby(by=['id','can'])
    mb = master_bar(gprs)

    for (uid, can), group in mb:   
        binary_string_length = group.binary.str.len().max()
        batch_size = len(group)

        pb = progress_bar(group.binary, parent=mb)
        pb.comment = "Converting bits string to bits array"
        chrs = [list(c) for c in pb]

        pb = progress_bar(range(len(chrs)), parent=mb)
        pb.comment = "Converting bits array to numeric"
        for i in pb:
            chrs[i] = pd.to_numeric(chrs[i])

        tmp_dict = {
            "id": uid,
            "can": can
        }
        tmp_dict_mag = {
            "id": uid,
            "can": can
        }

        #print("vstack")
        tmp = np.vstack(chrs)
        #display(tmp[1:5,:])

        pb = progress_bar(range(binary_string_length), parent=mb)
        pb.comment = "Calculating bitflips"
        for c in pb:
            tmpC = tmp[:,c]
            #print(''.join(map(str, tmpC)))
            tmpS = np.sum(np.abs(np.diff(tmpC)))
            #print("Sum", tmpS)
            tmp_dict[c] = tmpS / batch_size
            try:
                # Beware that in READ (Marchetti2019) the function is math.ceiling not math.floor
                # However, as described in our article, this is not affecting the results.
                tmp_dict_mag[c] = int(math.floor(math.log10(tmpS / batch_size)))
            except Exception as ex:
                tmp_dict_mag[c] = float('-inf')

        #print(uid, can, batch_size, tmp.shape, ''.join(map(str, tmp_dict.values())))
        
        bitflips = bitflips.append(tmp_dict, ignore_index=True)
        magnitudes = magnitudes.append(tmp_dict_mag, ignore_index=True)

    return bitflips, magnitudes

bitflips, magnitudes = create_bitflips(df)#.head(100))

In [ ]:
#display(bitflips.info())
display(bitflips.head())
#display(magnitudes.info())
display(magnitudes.head())

In [ ]:
%%time
# Save the intermediate result as Pickle file to speedup future analysis
#bitflips.to_pickle(DATA_DIR + "raw.bitflips.pkl")
#magnitudes.to_pickle(DATA_DIR + "raw.magnitudes.pkl")

## Data field decode


In [ ]:
# Should always load the binary extended with the interarrival packet time
#df = pd.read_pickle(DATA_DIR + "raw.pkl")
#df = pd.read_pickle(DATA_DIR + "raw.interrarrivaltimes.pkl")

# Load the PKL files instead of re-executing the analysis.
#bitflips = pd.read_pickle(DATA_DIR + "raw.bitflips.pkl")
#magnitudes = pd.read_pickle(DATA_DIR + "raw.magnitudes.pkl")

In [ ]:
display(df.head(2))
display(bitflips.head(2))
display(magnitudes.head(2))

### Data statistics

Calculate the lower and upper bounds of the time series. It will be used for graphical purposes.

In [ ]:
%%time
xlim_min = min(df.time)
xlim_max = max(df.time)
print("Time limits:", xlim_min, xlim_max)

### Datablocks analysis

This analysis converts the binary data frames into a unified data format where there is a value for each instant, ID, CANline and variable trace.

#### Identify the datablocks boundaries
It cycles over the dataframe `bitflips` that presents a row for each `ID` and `CANline`. The algorithm implements the Phase1 algorithm of [Marchetti2017](http://dx.doi.org/10.1109/TIFS.2018.2870826).

The intended output is a dictionary of datablocks to be analysed (`raw_datablocks`).

In [ ]:
raw_datablocks = {}

# Cicle over the dataframe that contains the bitflips.
for idx, row in bitflips.iterrows():
    # Bitflips list, remove both ID and CANline
    b = list(bitflips.loc[idx,:])
    uid = b[0]
    can = b[1]
    del b[0]
    del b[0]
    
    try:
        limit = list(map(tuple, np.where(np.isnan(b))))[0][0]
    except:
        limit = len(b)
        
    # If there are no bitflips in the dataset, provide a placeholder value
    if not np.asarray(b).any():
        raw_datablocks[(uid, can)] = None
        continue
    
    # Datablocks extracted from the current ID-CANline
    current_datablocks = []
    
    start_idx = 0
    end_idx = None
    while start_idx < limit:
        #print("Main", uid, can, start_idx, end_idx)
        while start_idx < limit and b[start_idx] == 0:
            start_idx += 1
            #print(" - SKIP START", start_idx, end_idx)
        end_idx = start_idx
        while end_idx < limit:
            #print(" - CHECK END ", start_idx, end_idx)
            if not np.isfinite(b[end_idx]):
                #print(" - - End Seq.\t(", start_idx, ",", end_idx-1, ")")
                current_datablocks.append({'start_idx': start_idx, 'end_idx': end_idx -1})
                end_idx = limit
                start_idx = limit
                break
            elif b[end_idx] == 0:
                #print(" - - B is 0\t(", start_idx, ",", end_idx-1, ")")
                current_datablocks.append({'start_idx': start_idx, 'end_idx': end_idx -1})
                start_idx = end_idx -1
                break
            elif end_idx == limit -1:
                #print(" - - End Seq.\t(", start_idx, ",", end_idx, ")")
                current_datablocks.append({'start_idx': start_idx, 'end_idx': end_idx})
                start_idx = end_idx
                break
            
            end_idx += 1
        start_idx += 1
    raw_datablocks[(uid, can)] = current_datablocks
    print(uid, can, current_datablocks)

In [ ]:
%%time
# Save the intermediate result as Pickle file to speedup future analysis
#with open(DATA_DIR + 'raw.rawdatablocks.pkl', 'wb') as handle:
#    pickle.dump(raw_datablocks, handle, protocol=pickle.HIGHEST_PROTOCOL)

#### Recognize data types
It cycles over the datablocks identified in the previous phase (`raw_datablocks`), the bitflips and the magnitudes. 

The algorithm implements the Phase2 algorithm of [Marchetti2017](http://dx.doi.org/10.1109/TIFS.2018.2870826).

Use the flag `VERBOSE` to enable extra logging

In [ ]:
VERBOSE = False

# Create the UID,CANline groups
bitflips_grp = bitflips.groupby(by=['id', 'can'])
magnitudes_grp = magnitudes.groupby(by=['id', 'can'])

# Final dictionary with all variables
datablocks = {}

In [ ]:
#with open(DATA_DIR + 'raw.rawdatablocks.pkl', 'rb') as handle:
#    raw_datablocks = pickle.load(handle)

The function separate the raw datablocks according to the magnitude heuristic ([Marchetti2017](http://dx.doi.org/10.1109/TIFS.2018.2870826)).

The intended return of the method consists of two variables:
 - the `processed_datablocks` variable that includes the final variable split.
 - the `remaining_blocks` collection that includes the datablocks that still require further analysis.

In [ ]:
def process_raw_datablocks(uid, can, raw_datablocks, b, m):
    current_raw_datablocks = raw_datablocks
    
    processed_datablocks = {
        'binary': [],
        'counter': [],
        'crc': [],
        'nibble': [],
        'byte': [],
        'halfword': [],
        'word': [],
    }
    
    # Skip the ID-CANline if there are no datablocks
    # a.k.a all the bits in all the frames are constant.
    if not current_raw_datablocks:
        if VERBOSE:
            print("\t  SKIP")
        return processed_datablocks, []
        
    # Blocks still to be analysed
    remaining_blocks = []
    
    # For each datablock
    for datablock in current_raw_datablocks:
        if VERBOSE:
            print("\t" + str(datablock))
            
        # Binary datablocks are easy to separate from the others
        if datablock['start_idx'] == datablock['end_idx']:
            if VERBOSE:
                print("\t  BINARY", datablock)
            processed_datablocks['binary'].append(datablock)
            continue
        
        # If matching the conditions split the datablock
        start = datablock['start_idx']
        for end in range(datablock['start_idx'], datablock['end_idx'] + 1):
            
            #DEBUG ONLY
            #if VERBOSE:
            #    print("- - Checking from ", start, "to", end)
            #    pass
            
            # Skip the very first check, ie when end == start
            # Not sure that this is the best way to do it, but the algorithm in Marchetti2017 says so.
            if end != datablock['start_idx']:
                
                # End reached the last bit of the datablock. Close it.
                if end == datablock['end_idx']:
                    if VERBOSE:
                        #DEBUG ONLY
                        #print("- - End of block")
                        print("\t  DATABLOCK", {'start_idx': start, 'end_idx': datablock['end_idx']})
                    remaining_blocks.append({'start_idx': start, 'end_idx': datablock['end_idx']})
                    
                # There is a change of magnitude. Close the block.
                elif m[end] < m[oldindex]: 
                    if VERBOSE:
                        #DEBUG ONLY
                        #print("- - Change of magnitude")
                        print("\t  DATABLOCK", {'start_idx': start, 'end_idx': end-1})
                        
                    remaining_blocks.append({'start_idx': start, 'end_idx': end-1})
                    start = end
                
                #DEBUG ONLY    
                #else:
                #    if VERBOSE:
                #        print("- - Not end and not change of magnitude. go on")
                #        pass
            
            #DEBUG ONLY
            #else:
            #    if VERBOSE:
            #        print("- - end == start SKIP")
            #        pass
                    
            oldindex = end
            
    # Returns the recognized values so far, and the remaining blocks to be analysed.
    return processed_datablocks, remaining_blocks

The function takes the remaining blocks and recognise their type ([Marchetti2017](http://dx.doi.org/10.1109/TIFS.2018.2870826)).

The intended return of the method consists of variable:
 - the `processed_datablocks` variable that includes the final variable split.

In [ ]:
def process_remaining_datablocks(uid, can, processed_datablocks, remaining_blocks, b, m):
    
    # 1% epsilon
    EPSILON = 0.01
    
    if VERBOSE:
        print("- Checking DATABLOCKS")
        
    for block in remaining_blocks:
        #pb.Comment = block
        if VERBOSE:
            print("\t" + str(block))
        
        # Check if it is a COUNTER
        if block['start_idx'] != block['end_idx']:
            
            # magnitude of the least significant bit = 0 <-> bitiflip = 1
            if b[block['end_idx']] == 1: 
                init = block['end_idx'] - 1
                end_idx = block['end_idx']
                while init >= block['start_idx']:
                    # the bitflip rate should double at each step from the most 
                    # significant bit to the least significant
                    if math.isclose(2*b[init], b[init+1], rel_tol=EPSILON):
                        init = init - 1
                    else:
                        break

                # Save this block as counter
                if VERBOSE:
                    print("\t  COUNTER", {'start_idx': init + 1, 'end_idx': end_idx})
                    
                processed_datablocks['counter'].append({'start_idx': init + 1, 'end_idx': end_idx})

                block['end_idx'] = init
            else:
                if VERBOSE:
                    print("\t  Not a COUNTER")
        else:
            if VERBOSE:
                print("\t  BINARY", block)
            processed_datablocks['binary'].append(block)
        # END COUNTER
        
        # Discard all the blocks that have negative length after counter analysis
        start_idx = block['start_idx']
        end_idx = block['end_idx']
        if start_idx < block['end_idx']:  
            
            out = False
            for i in range(start_idx, end_idx):
                exit = False
                idx = b[i:end_idx + 1]
                if VERBOSE:
                    msg = "\t  Checking b["+str(i)+" : "+str(end_idx)+"+1]"
                
                # Check if the values in the datablock fit a normal distribution
                # centered in 0.5 (so 0.5 ± std)
                mean = statistics.mean(idx)
                std = statistics.stdev(idx)
                if (0.5 - std <= mean) and (mean <= 0.5 + std):
                    for j in range(i, int(b[1] + 1)):
                        # Beware, in READ algorithm this condition was different
                        # but we are using floor instead of ceiling, so at the end 
                        # there are no changes
                        
                        if m[j] < -1: 
                            exit = True
                    if exit is True:
                        break
                    else:
                        if VERBOSE:
                            print("\t  CRC", {'start_idx': i, 'end_idx': end_idx})
                        processed_datablocks['crc'].append({'start_idx': i, 'end_idx': end_idx})
                        end_idx = i - 1
                        out = True
                        if start_idx <= end_idx:
                            if VERBOSE:
                                print("\t  Reinject block", {'start_idx': start_idx, 'end_idx': end_idx})
                            remaining_blocks.append({'start_idx': start_idx, 'end_idx': end_idx})
                else:
                    if VERBOSE:
                        print(msg + " - Not a CRC") 
                if out:
                    break
            if not out:
                size = block['end_idx'] - start_idx
                if size <= 4:
                    blocktype = "nibble"
                elif size <=8:
                    blocktype = "byte"
                elif size <=16:
                    blocktype = "halfword"
                else:
                    blocktype = "word"
                        
                if VERBOSE:
                    print("\t  " + blocktype.upper(), {'start_idx': start_idx, 'end_idx': block['end_idx']})
                processed_datablocks[blocktype].append({'start_idx': start_idx, 'end_idx': block['end_idx']})
        else:
            if VERBOSE:
                print("\t  SKIP (block has now negative size)")    
    return processed_datablocks

Execute the analysis and extract all the variables from the datablocks

In [ ]:
def process_trace(uid, can, bitflips_grp, magnitudes_grp):
    current_raw_datablocks = raw_datablocks[(uid, can)]
    
    current_bitflips_grp = bitflips_grp.get_group((uid, can))
    current_bitflips_grp.reset_index(inplace=True, drop=True)
    current_bitflips_grp = list(current_bitflips_grp.loc[0,:])
    del current_bitflips_grp[0]
    del current_bitflips_grp[0]
    
    current_magnitudes_grp = magnitudes_grp.get_group((uid, can))
    current_magnitudes_grp.reset_index(inplace=True, drop=True)
    current_magnitudes_grp = list(current_magnitudes_grp.loc[0,:])
    del current_magnitudes_grp[0]
    del current_magnitudes_grp[0]
    
    processed_datablocks, remaining_blocks = process_raw_datablocks(uid, can,
                                                                    current_raw_datablocks, 
                                                                    current_bitflips_grp, 
                                                                    current_magnitudes_grp)
    processed_datablocks = process_remaining_datablocks(uid, can,
                                                        processed_datablocks, 
                                                        remaining_blocks, 
                                                        current_bitflips_grp,
                                                        current_magnitudes_grp)
    return processed_datablocks

In [ ]:
VERBOSE = False
mb = progress_bar(raw_datablocks)
for key in mb:
    mb.comment = key
    if VERBOSE:
        print(key, "-----------------------------")
    
    # Finally save the resulted datablocks
    datablocks[key] = process_trace(key[0], key[1], bitflips_grp, magnitudes_grp)

In [ ]:
%%time
# Save the intermediate result as Pickle file to speedup future analysis
#with open(DATA_DIR + 'raw.datablocks.pkl', 'wb') as handle:
#    pickle.dump(datablocks, handle, protocol=pickle.HIGHEST_PROTOCOL)

#### Extract values from DATA and BINARY Blocks

The function converts a ID/CANline trace to a collection of variables. 
In the associated paper, this is defined as `unified` data.

In [ ]:
%%time
with open(DATA_DIR + 'raw.datablocks.pkl', 'rb') as handle:
    datablocks = pickle.load(handle)

In [ ]:
%%time
def extract_variables_values(uid, can, df, datablocks, datatypes = ['binary', 'nibble', 'byte', 'halfword','word']):
    rows = []
    for row in df.itertuples():
        bits = row.binary
        
        for datatype in datatypes:
            # Data
            n = 0
            for datablock in datablocks[datatype]:
                variable = bits[datablock['start_idx']:datablock['end_idx']+1]
                rows.append({
                    'time': row.time,
                    'id': uid,
                    'can': can,
                    'datatype': datatype,
                    'variable' : datatype[0:2].upper() + "_" + str(n),
                    'value' : int(variable, base=2)
                })
                n += 1
            
    return rows

In [ ]:
%%time
grps = df.groupby(by=["id","can"])
uid, can = next(iter(grps.groups))

res = extract_variables_values(uid, can, grps.get_group((uid, can)), datablocks[(uid, can)])

In [ ]:
%%time
pd.DataFrame(res).head(5)

Calculates the variables foreach trace

In [ ]:
%%time
unified = []
pb = progress_bar(df.groupby(by=["id","can"]))
for (uid, can), group in pb:
    pb.comment = uid + " - " + can
    variables = extract_variables_values(uid, can, group, datablocks[(uid, can)])
    unified.extend(variables)

In [ ]:
%%time
unified = pd.DataFrame(unified)

display(unified.info())
display(unified.sample(10))

In [ ]:
%%time
with open(DATA_DIR + 'unified.pkl', 'wb') as handle:
    pickle.dump(unified, handle, protocol=pickle.HIGHEST_PROTOCOL)

#unified.to_csv(DATA_DIR + 'unified.csv', sep=",", index=False)

## Export figures

### Packet count per ID and CANline

In [ ]:
%%time
from matplotlib.ticker import ScalarFormatter

def packet_count_per_id_and_canline(df, show=True, save=True):  
    grp = df.groupby(['can','id']).agg(['count'])['binary'].reset_index()
    grp['log'] = np.log10(grp['count'])
    grp.head()

    var = 'log'
    palettes = {
        'can0': "Blues_r",
        'can1': 'Greens_r'
    }

    for can in grp.can.unique():
        sns.set(font_scale=1.2)
        sns.set_style("whitegrid")
        sns.set_style({'font.family':'monospace'})

        sub = grp[grp.can==can].copy()
        sub.sort_values(by=var, ascending=False, inplace=True)
        sub.reset_index(drop=True, inplace=True)

        fig = plt.figure(figsize=(15,5))
        ax = sns.barplot(data=sub, x='id', y=var, order=sub['id'], palette=palettes[can])
        plt.xticks(rotation='vertical')
        plt.ylim(0,max(sub[var])+1)
        #[x.set_va('top') for x in ax.get_xticklabels()]
        if var == 'log':
            t = ['%.0E' % (10**y) for y in ax.get_yticks()]
            ax.set_yticklabels(t)
            ax.set(xlabel="ID CODE", ylabel="Number of packets (log scale)")
        else:
            ax.set(xlabel="ID CODE", ylabel="Number of packets")

        #plt.legend(loc="upper right", title="CAN line")
        plt.tight_layout()
        if save:
            plt.savefig(GRAPHICS_DIR + var + "-" + can + "." + FIG_EXTENSION)
        if show:
            plt.show()
        plt.close()
        
#packet_count_per_id_and_canline(df, show=True, save=True)

### Interrarrival packet time distributions and boxplots

#### Individual distributions

In [ ]:
%%time
def formatter(value, tick_number):
    return "{:.2f}".format(value).rjust(8)

def timedeltas(data, uid, xlim_min, xlim_max, show=True, save=True):
    
    
    grps = data.groupby(by='can')
    fig = plt.figure(figsize=(15, 1 + 2*len(grps)),
                     constrained_layout=True,)
    
    fig.suptitle("Identifier: " + uid, y=1, x=0, ha='left', fontsize='large', fontweight='bold')
    
    spec = gridspec.GridSpec(ncols=2, 
                             nrows=len(grps), 
                             figure=fig,
                             width_ratios = [10,2],
                             hspace=1,
                             wspace=0
                            )
    
    #fig.suptitle('Interarrival for ID ' + str(uid), y=1.05)
    sns.set_style("whitegrid")
    sns.set_style({'font.family':'monospace'})
    
    palettes = {
        "can0": '#4C72B0',
        "can1": '#C44E52'
    }

    ax_idx = 0
    for can, d in grps:    
        if (len(d) < 1):
            continue
        
        lax = fig.add_subplot(spec[ax_idx, 0])
        lax = sns.scatterplot(data=d, x=d.time, y=d.timedelta_ms, ax=lax, color=palettes[can])
        lax.xaxis.grid(False)
        lax.set_ylabel('Timedelta in ms')
        lax.set_xlabel('Packet arrival time for (' + uid + ', ' + can + ')')
        lax.set_xlim(left=xlim_min, right=xlim_max)
 
        lax.yaxis.set_major_formatter(plt.FuncFormatter(formatter))
    
        rax = fig.add_subplot(spec[ax_idx, 1], sharey=lax)
        rax = sns.boxenplot(data=d, 
                            y=d.timedelta_ms, 
                            color=palettes[can],
                            ax=rax)
        rax.set(xlabel=can, ylabel=None, title="Boxplots")
        rax.yaxis.set_major_formatter(plt.FuncFormatter(formatter))
        
        ax_idx += 1
    #plt.tight_layout()

    if save:
        try:
            os.makedirs(GRAPHICS_DIR + "timedeltas/")
        except FileExistsError:
            # directory already exists
            pass
        plt.savefig(GRAPHICS_DIR + "timedeltas/timedelta-" + uid + "." + FIG_EXTENSION, bbox_inches = "tight")
    if show:
        plt.show()
        
    plt.close()

#test_id = next(iter(df.id.unique()))
#timedeltas(uid=test_id, show=True, save=False,
#           xlim_min = xlim_min,
#           xlim_max = xlim_max,
#           data=df.groupby('id').get_group(test_id)#.sample(1000)
#          )

Calculate the interarrival time for each UID.

In [ ]:
for uid, group in progress_bar(df.groupby(by='id')):
    try:
        timedeltas(data=group, 
                   uid=uid, 
                   xlim_min=xlim_min,
                   xlim_max=xlim_max,
                   show=False, 
                   save=True)
    except Exception as ex:
        print("Exception with ", uid, ex)

print("Done")

#### Overall view of vehicle

In [ ]:
%%time
def timedeltaboxes(data, 
                   show=True, 
                   save=False, 
                   log = False, 
                   fliers = [True, True]
                  ):
    import matplotlib as mpl
    palettes = {
        "can0": '#4C72B0',
        "can1": '#C44E52'
    }
       
    fig = plt.figure(figsize=(15,3), constrained_layout=True)
    sns.set_style("whitegrid")
    sns.set_style({'font.family':'monospace'})
    
    spec = gridspec.GridSpec(ncols=2, 
                             nrows=1, 
                             figure=fig,
                             width_ratios = [10,2],
                             hspace=1,
                             wspace=0
                            )
    
    lax = fig.add_subplot(spec[0, 0])
    lax = sns.boxplot(data=data, 
                      x='id', 
                      hue='can', 
                      y='timedelta_ms', 
                      ax=lax,
                      palette=palettes, 
                      showfliers=fliers[0]
                     )
    plt.xticks(rotation='vertical')
    lax.set_ylabel('Timedelta in ms')
    lax.set_xlabel('IDs')

    plt.legend(loc="best", ncol=2, title="CAN line")
    #plt.tight_layout()
    
    rax = fig.add_subplot(spec[0, 1], sharey=lax)
    rax = sns.boxenplot(data=data, 
                        x='can', 
                        y='timedelta_ms', 
                        palette=palettes,
                        ax=rax,
                        linewidth=0,
                        #showfliers=fliers[2]
                       )
    rax.set_ylabel('')
    rax.set_xlabel('CAN lines')

    if log:
        plt.yscale('log')
        rax.get_yaxis().set_major_locator(mpl.ticker.LogLocator())
        rax.get_yaxis().set_major_formatter(mpl.ticker.LogFormatter())
        #rax.get_yaxis().set_minor_locator(mpl.ticker.LogLocator())
        #rax.get_yaxis().set_minor_formatter(mpl.ticker.LogFormatter())
    
    if save:
        plt.savefig(GRAPHICS_DIR + "interarrivalpackettime" + ("-log" if log else "") + 
                    "." + FIG_EXTENSION, bbox_inches='tight')
    if show:
        plt.show()
    plt.close()
    

#timedeltaboxes(show=True, 
#               save=False, 
#               log=False,
#               data=df#.head(1000)
#               )

Calculate the actual figures

In [ ]:
timedeltaboxes(show=True, 
               save=True, 
               fliers = [False, False],
               log=True,
               data=df#.sample(10000)
               )

### Bitflips heatmaps

#### Complete heatmap

In [ ]:
def bitflips_heatmap(results, 
                     show=True, 
                     save=False, save_filename_extra="",
                     row_height = 0.2, 
                     palette = sns.color_palette('gray_r', 5),
                     ylim_min = 0, ylim_max = 1, 
                     hlines=None, vlines=None):
    
    labels = results.id + ' ' + results.can
    
    fig = plt.figure(figsize=(15,row_height*len(results.index),))
    #fig = plt.figure(figsize=(15,3))
    sns.set_style("whitegrid")
    sns.set_style({'font.family':'monospace'})
    sns.set_context("notebook", font_scale=0.8, rc={"lines.linewidth": 2.5})
    ax = sns.heatmap(data=results[results.select_dtypes('number').columns], 
                     yticklabels=labels,
                     linewidths=.2, 
                     vmin=ylim_min, 
                     vmax=ylim_max, 
                     cmap=palette
                    )
    if vlines:
        for line in vlines:
            ax.vlines(x=line['x'], 
                      ymin=line['ymin'], 
                      ymax=line['ymax'], 
                      colors=line['colors'], 
                      linewidth = 2,
                      #linestyles='dotted', 
                      label=line.get('label'))
    if hlines:
        for line in hlines:
            ax.hlines(y=line['y'], 
                      xmin=line['xmin'], 
                      xmax=line['xmax'], 
                      colors=line['colors'], 
                      linewidth = 2,
                      #linestyles='dotted', 
                      label=line.get('label'))
    
    ax.xlabel = "Bit position in the data field"
    ax.ylabel = "ID and CAN line"
    plt.yticks(rotation=0) 
    plt.tight_layout()
    
    # fix for mpl bug that cuts off top/bottom of seaborn viz
    b, t = plt.ylim() # discover the values for bottom and top
    b += 0.5 # Add 0.5 to the bottom
    t -= 0.5 # Subtract 0.5 from the top
    plt.ylim(b, t) # update the ylim(bottom, top) values
    
    if vlines or hlines:
        plt.legend(loc='lower center', 
                   ncol=7,
                   bbox_to_anchor=(0.5, 1),
                   title='Var. Type'
                  )
    
    
    if save:
        filename = GRAPHICS_DIR + "bitflips-heatmap" + save_filename_extra + ("-annotated" if hlines or vlines else "") + "." + FIG_EXTENSION
        plt.savefig(filename, bbox_inches = "tight")

    if show:
        plt.show()
    plt.close()
    
#bitflips_heatmap(bitflips.sample(10), 
#                 show=True, 
#                 save=False,
#                 row_height = 0.2, 
#                )

#### Create the heatmap with colored boxes

In [ ]:
def create_variable_boxes(datablocks):
    boxes_palette = {
        'binary': ['#DB5E56'],
        'counter': ['#91DB56'],
        'crc': ['#B15928'],
        'nibble': ['#DBC256'],
        'byte': ['#0F8935'],
        'halfword': ['#566FDB'],
        'word': ['#DB56B2'],
    }
    labels = {
        'binary': False,
        'counter': False,
        'crc': False,
        'nibble': False,
        'byte': False,
        'halfword': False,
        'word': False,
    }
    
    boxes = [
        # Prototype:
        #{'x': 0,   'y': 0,   'w': 3,   't': 'short'},
    ]
    
    idx = 0
    for key, boxes_dict in datablocks.items():
        for var_type, var_type_boxes in boxes_dict.items():
            for box in var_type_boxes:
                boxes.append(
                    {'x': box['start_idx'],
                     'y': idx,
                     'w': 1 + box['end_idx'] - box['start_idx'],
                     't': var_type}
                )
        idx += 1

    vlines = []
    hlines = []

    for box in boxes:
        vlines.append({'x': box['x'],
                       'ymin': box['y'],
                       'ymax': box['y'] +1,
                       'colors': boxes_palette[box['t']], 
                       'label': box['t'].capitalize() if not labels[box['t']] else None,
                      })
        labels[box['t']] = True

        vlines.append({'x': box['x'] + box['w'],
                       'ymin': box['y'],
                       'ymax': box['y'] +1,
                       'colors': boxes_palette[box['t']], 
                       'label': box['t'].capitalize() if not labels[box['t']] else None,
                      })

        hlines.append({'y': box['y'],
                       'xmin': box['x'],
                       'xmax': box['x'] + box['w'],
                       'colors': boxes_palette[box['t']], 
                       'label': box['t'].capitalize() if not labels[box['t']] else None,
                      })
        hlines.append({'y': box['y'] + 1,
                       'xmin': box['x'],
                       'xmax': box['x'] + box['w'],
                       'colors': boxes_palette[box['t']], 
                       'label': box['t'].capitalize() if not labels[box['t']] else None,
                      })
    
    return vlines, hlines

#### FIlter by ID and create the heatmap

In [ ]:
%%time
magnitudes = pd.read_pickle(DATA_DIR + "raw.magnitudes.pkl")
with open(DATA_DIR + 'raw.datablocks.pkl', 'rb') as handle:
    datablocks = pickle.load(handle)

ids = list(df.id.unique())
filtered = magnitudes[magnitudes.id.isin(ids).values]
#filtered = magnitudes
display(filtered.head())

tmp = filtered.select_dtypes(include=np.number)
min_scale = int(np.around(np.abs(np.nanmin(tmp[tmp != -np.inf]))))
x_loc = 0 # replace this with the X coordinate of a -Inf value
y_loc = 0 # replace this with the Y coordinate of a -Inf value
filtered = filtered.replace(filtered.loc[x_loc,y_loc], -1 -min_scale) #Replace the infinite values with min - 1
display(filtered.head())
grps = filtered.groupby(by=['id','can'])

filtered_datablocks = { key: datablocks[key] for key in grps.groups.keys() }
#display(filtered_datablocks)

vlines, hlines = create_variable_boxes(filtered_datablocks)

ROW_HEIGHT = 0.2

bitflips_heatmap(filtered, 
                 show=True, 
                 save=True, save_filename_extra='-magnitude',
                 row_height = ROW_HEIGHT, 
                 palette = ["#E9EBEB","#D0D2D2","#B6B8B8","#838585","#505252"], #Light to dark
                 #palette = ["#505252","#838585","#B6B8B8","#D0D2D2","#E9EBEB"], # Dark to light
                 ylim_min = -5, ylim_max = 0, 
                 #hlines=hlines, vlines=vlines
                )

bitflips_heatmap(filtered, 
                 show=True, 
                 save=True, save_filename_extra='-magnitude',
                 row_height = ROW_HEIGHT, 
                 palette = ["#E9EBEB","#D0D2D2","#B6B8B8","#838585","#505252"], #Light to dark
                 #palette = ["#505252","#838585","#B6B8B8","#D0D2D2","#E9EBEB"], # Dark to light
                 ylim_min = -5, ylim_max = 0, 
                 hlines=hlines, vlines=vlines
                )


bitflips = pd.read_pickle(DATA_DIR + "raw.bitflips.pkl") 
filtered = bitflips[bitflips.id.isin(ids).values]
bitflips_heatmap(filtered, 
                 show=True, 
                 save=True, save_filename_extra='-percentage',
                 row_height = ROW_HEIGHT, 
                 palette = ["#E9EBEB","#D0D2D2","#B6B8B8","#838585","#505252"], #Light to dark
                 #palette = ["#505252","#838585","#B6B8B8","#D0D2D2","#E9EBEB"], # Dark to light
                 #ylim_min = -5, ylim_max = 0, 
                 #hlines=hlines, vlines=vlines
                )

In [ ]:
bitflips = pd.read_pickle(DATA_DIR + "raw.bitflips.pkl")
#magnitudes = pd.read_pickle(DATA_DIR + "raw.magnitudes.pkl")

vlines, hlines = create_variable_boxes(datablocks)

bitflips_heatmap(bitflips, 
                 show=True, 
                 save=False, #save_filename_extra='-' + ('-'.join(magnitudes.id.unique())),
                 row_height = 0.2, 
                 #palette = sns.color_palette('gray_r', 5),
                 palette = ["#E9EBEB","#D0D2D2","#B6B8B8","#838585","#505252"], #Light to dark
                 #palette = ["#505252","#838585","#B6B8B8","#D0D2D2","#E9EBEB"], # Dark to light
                 ylim_min = 0, ylim_max = 0.1, 
                 hlines=hlines, vlines=vlines)